In [ ]:
import os

project_folder = "/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/claude_results"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(project_folder) == False:
    os.mkdir(project_folder)
    print(project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(project_folder)
create_and_set_working_directory(project_folder)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data_diseases=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:
test_data=test_data_diseases[['id', 'selected_posts']]

In [ ]:
all_data = pd.concat([train_data, val_data], axis=0, ignore_index=True)
all_data = pd.concat([all_data, test_data_diseases], axis=0, ignore_index=True)

In [ ]:
y_all=[]
for row in all_data['diseases']:
  if 'bipolar' in row:
    y_all.append(1)
  else:
    y_all.append(0)

In [ ]:
y_true_train=[]
for row in train_data['diseases']:
  if 'schizophrenia' in row:
    y_true_train.append(1)
  else:
    y_true_train.append(0)

In [ ]:
y_true_test=[]
for row in test_data_diseases['diseases']:
  if 'schizophrenia' in row:
    y_true_test.append(1)
  else:
    y_true_test.append(0)

In [ ]:
count=0
for row in y_all:
  if row==1:
    count+=1
print(count)

345


In [ ]:
count=0
for row in y_true_test:
  if row==1:
    count+=1
print(count)

17


### **Anxiety**

In [ ]:
gad7_keywords = [
    ["nervous", "anxious", "on edge"],
    ["can't stop worrying", "can't control worrying"],
    ["worrying too much", "excessive worrying"],
    ["trouble relaxing"],
    ["restless", "hard to sit still"],
    ["easily annoyed", "irritable"],
    ["feeling afraid", "something awful might happen"]
]

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

!python -m nltk.downloader stopwords
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Train the model
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# Perform cross-validation
model.fit(X_train, y_train)
predictions=model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Standard Deviation of F1 Scores (Bootstrap): 0.03334622718882116
Standard Deviation of Recall Scores (Bootstrap): 0.03191497895545451
Standard Deviation of Precision Scores (Bootstrap): 0.004052742653125046
f1_score 0.464846516933431
recall 0.35069747676628277
precision 0.9579844326095862
std_f1_score 0.03334622718882116
std_recall_score 0.03191497895545451
std_precision_score 0.004052742653125046


In [ ]:
with open('anxiety-test-simple.npy', 'wb') as f:
    np.save(f, predictions)

In [ ]:
#baseline+all
!pip install -U scikit-learn
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.3917 ± 0.0607
Sensitivity: 0.3024 ± 0.0481
Specificity: 0.9689 ± 0.0539


In [ ]:
with open('anxiety-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#simple+all
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_true_train


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.3917 ± 0.0607
Sensitivity: 0.3024 ± 0.0481
Specificity: 0.9689 ± 0.0539


In [ ]:
import joblib

joblib.dump(rf_classifier, "random_forest_classifier.joblib")

['random_forest_classifier.joblib']

In [ ]:
prediction_test=rf_classifier.predict(X_combined[100:200])

In [ ]:
f1_score(y_true_train[100:200], prediction_test)

0.019801980198019802

In [ ]:
with open('y_test_best.npy', 'wb') as f:
    np.save(f, y_test)

In [ ]:
from scipy import sparse

#Save
sparse.save_npz('X_test_best.npz', X_test)

In [ ]:
#!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 3)  # Limit the rating to a maximum of 3 based on GAD-7 scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, gad7_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the presence of anxiety based on a threshold of 5
anxiety_threshold = 5
X_anxiety = np.where(np.sum(X_ratings, axis=1) >= anxiety_threshold, 1, 0)

# Combine text features and anxiety presence
X_combined = hstack([X_text_vec, X_anxiety.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

X_combined = X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

predictions = rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100
# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
  indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
  bootstrap_true_labels = np.take(true_labels, indices)
  bootstrap_preds = np.take(preds, indices)
  f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
  recall = recall_score(bootstrap_true_labels, bootstrap_preds)
  precision = precision_score(bootstrap_true_labels, bootstrap_preds)
  return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)
# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Standard Deviation of F1 Scores (Bootstrap): 0.002473165079025267
Standard Deviation of Recall Scores (Bootstrap): 0.0008199850904003263
Standard Deviation of Precision Scores (Bootstrap): 0.0043503644081515054
f1_score 0.9565598673397717
recall 0.9983846546463291
precision 0.9181078524116326
std_f1_score 0.002473165079025267
std_recall_score 0.0008199850904003263
std_precision_score 0.0043503644081515054


In [ ]:
with open('path/to/file', 'wb') as f:
    cPickle.dump(rf, f)

In [ ]:
predictions=np.load('anxiety-test-assessment.npy')

In [ ]:
#assessment+ test
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack
nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        if any(item.lower() in post.lower() for post in user_posts):
            user_ratings.append(1)  # Item is present in user's posts
        else:
            user_ratings.append(0)  # Item is not present in user's posts
    return user_ratings

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_text = vectorizer.fit_transform(all_data['selected_posts'])

X_ratings = np.zeros((len(all_data), len(gad7_keywords)))
for i, item in enumerate(gad7_keywords):
    X_ratings[:, i] = all_data['selected_posts'].apply(lambda x: item.lower() in x.lower()).astype(int)

# Combine text features and rating features using sparse matrices
X_combined = hstack((X_text, X_ratings))

y = y_true_train

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score, average='weighted'),
    'sensitivity': make_scorer(recall_score, average='weighted'),
    'specificity': make_scorer(specificity_score)
}

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
predictions=model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, sensitivity_scores, specificity_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_sensitivity_score= np.std(sensitivity_scores)
std_specificity_score = np.std(specificity_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of sensitivity Scores (Bootstrap):", std_sensitivity_score)
print("Standard Deviation of specificity Scores (Bootstrap):", std_specificity_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("sensitivity", np.mean(sensitivity_scores))
print("specificity", np.mean(specificity_scores))
print("std_f1_score", std_f1_score)
print("std_sensitivity_score", std_sensitivity_score)
print("std_specificity_score", std_specificity_score)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'list' object has no attribute 'lower'

In [ ]:
with open('anxiety-test-assessment.npy', 'rb') as f:
    predictions=np.load('anxiety-test-assessment.npy')

In [ ]:
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
import nltk
nltk.download('stopwords')

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    recall = recall_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    precision = precision_score(bootstrap_true_labels, bootstrap_preds, average='weighted')
    return f1, recall, precision

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_text = vectorizer.fit_transform(all_data['selected_posts'])
X_ratings = np.zeros((len(all_data), len(gad7_keywords)))
for i, item in enumerate(gad7_keywords):
    X_ratings[:, i] = all_data['selected_posts'].apply(lambda x: any(keyword.lower() in x.lower() for keyword in item)).astype(int)

# Combine text features and rating features using sparse matrices
X_combined = hstack((X_text, X_ratings))
y = y_all

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X_combined, y, cv=5, scoring=scoring, return_estimator=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cross-validation scores:
F1 Score: 0.3873 ± 0.0611
Sensitivity: 0.2983 ± 0.0495
Specificity: 0.9688 ± 0.0543


In [ ]:
with open('anxiety-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

### **Bipolar**

In [ ]:
# Define the bipolar disorder assessment items from the checklist
bipolar_items = [
    "Feeling low, irritable or appearing tearful",
    "Reduced interest or pleasure in most activities",
    "Marked increase or decrease in appetite, or significant weight loss",
    "Difficulties sleeping and waking",
    "Increased agitation or lethargy",
    "Fatigue",
    "Feeling worthless or guilty",
    "Less able to make decisions or concentrate",
    "Recurrent thoughts of death or suicide",
    "Inflated self-esteem and grandiosity",
    "Decreased need for sleep",
    "More talkative than usual",
    "Flight of ideas or racing thoughts",
    "Distractibility",
    "Increase in goal-directed activity",
    "Excessive involvement in risky activities (e.g. overspending, sexual indiscretions, foolish investments)"
]

In [ ]:
# Define the bipolar disorder assessment keywords
bipolar_keywords = [
    "low mood",
    "irritability",
    "tearfulness",
    "loss of interest",
    "reduced pleasure",
    "appetite changes",
    "weight changes",
    "sleep disturbances",
    "agitation",
    "lethargy",
    "fatigue",
    "worthlessness",
    "guilt",
    "indecisiveness",
    "concentration difficulties",
    "suicidal thoughts",
    "inflated self-esteem",
    "grandiosity",
    "decreased need for sleep",
    "talkativeness",
    "flight of ideas",
    "racing thoughts",
    "distractibility",
    "goal-directed activity",
    "risky behavior"
]

In [ ]:
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_train = vectorizer.fit_transform(train_data['selected_posts'])
X_test = vectorizer.transform(test_data['selected_posts'])
y_train = y_true_train  # Assuming 'anxiety' is the column containing the anxiety labels in the training data
y_test = y_true_test  # Assuming 'anxiety' is the column containing the anxiety labels in the test data

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and specificity scores
def bootstrap_scores(true_labels, preds):
    tn, fp, fn, tp = confusion_matrix(true_labels, preds).ravel()
    f1 = f1_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    specificity = tn / (tn + fp)
    return f1, recall, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, y_pred) for _ in range(num_bootstraps)]

# Calculate mean and standard deviation of scores
f1_scores, recall_scores, specificity_scores = zip(*bootstrap_scores_list)
mean_f1_score = np.mean(f1_scores)
mean_recall_score = np.mean(recall_scores)
mean_specificity_score = np.mean(specificity_scores)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_specificity_score = np.std(specificity_scores)

# Print results in the desired format
print(f"F1 Score: {mean_f1_score:.4f}±{std_f1_score:.4f}")
print(f"Recall Score: {mean_recall_score:.4f}±{std_recall_score:.4f}")
print(f"Specificity Score: {mean_specificity_score:.4f}±{std_specificity_score:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-14-55aad6a57cac>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))


F1 Score: 0.4860±0.0000
Recall Score: 0.3333±0.0000
Specificity Score: 0.9976±0.0000


In [ ]:
with open('bipolar-test-simple.npy', 'wb') as f:
    np.save(f, y_pred)

In [ ]:
#baseline+all

!pip install -U scikit-learn

import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Preprocess the text data
X_text = vectorizer.fit_transform(all_data['selected_posts'])

# Generate assessment scale features
def generate_assessment_features(user_posts, assessment_items):
    user_features = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        user_features.append(count)
    return user_features

assessment_features = all_data['selected_posts'].apply(lambda x: generate_assessment_features(x, bipolar_keywords))
X_assessment = pd.DataFrame(assessment_features.tolist())

# Combine text features and assessment features
X = pd.concat([pd.DataFrame(X_text.toarray()), X_assessment], axis=1)

y = y_all

# Define the scoring metrics
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
with open('bipolar-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score, recall_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 1)  # Limit the rating to a maximum of 1 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data for train and test sets
stop_words = set(stopwords.words('english'))
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Convert text to Bag-of-Words representation for train and test sets
vectorizer = CountVectorizer()
X_train_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in train_data['selected_posts']])
X_test_text_vec = vectorizer.transform([' '.join(posts) for posts in test_data['selected_posts']])

# Generate ratings for each data point in train and test sets
X_train_ratings = []
for _, row in train_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, bipolar_keywords, vectorizer, stop_words)
    X_train_ratings.append(user_ratings)

X_test_ratings = []
for _, row in test_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, bipolar_keywords, vectorizer, stop_words)
    X_test_ratings.append(user_ratings)

# Convert ratings to numpy arrays for train and test sets
X_train_ratings = np.array(X_train_ratings)
X_test_ratings = np.array(X_test_ratings)

# Determine the presence of bipolar disorder based on the criteria from the checklist for train and test sets
manic_threshold = 3
depressive_threshold = 5
X_train_bipolar = np.where((np.sum(X_train_ratings[:, 9:], axis=1) >= manic_threshold) & (np.sum(X_train_ratings[:, :9], axis=1) >= depressive_threshold), 1, 0)
X_test_bipolar = np.where((np.sum(X_test_ratings[:, 9:], axis=1) >= manic_threshold) & (np.sum(X_test_ratings[:, :9], axis=1) >= depressive_threshold), 1, 0)

# Combine text features and bipolar presence for train and test sets
X_train_combined = hstack([X_train_text_vec, X_train_bipolar.reshape(-1, 1)])
X_test_combined = hstack([X_test_text_vec, X_test_bipolar.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
X_train_combined = X_train_combined.tocsr()
X_test_combined = X_test_combined.tocsr()

# Train the model
rf_classifier.fit(X_train_combined, y_true_train)
predictions = rf_classifier.predict(X_test_combined)

# ... (rest of the code remains the same)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, sensitivity, and specificity scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    tn, fp, fn, tp = confusion_matrix(bootstrap_true_labels, bootstrap_preds).ravel()
    specificity = tn / (tn + fp)
    return f1, sensitivity, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_true_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, sensitivity_scores, specificity_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_sensitivity_score = np.std(sensitivity_scores)
std_specificity_score = np.std(specificity_scores)

# Print results
print(f"F1 Score: {np.mean(f1_scores):.4f} ± {std_f1_score:.4f}")
print(f"Sensitivity: {np.mean(sensitivity_scores):.4f} ± {std_sensitivity_score:.4f}")
print(f"Specificity: {np.mean(specificity_scores):.4f} ± {std_specificity_score:.4f}")

F1 Score: 0.0246 ± 0.0185
Sensitivity: 0.0126 ± 0.0095
Specificity: 1.0000 ± 0.0000


In [ ]:
with open('bipolar-simple-assessment.npy', 'wb') as f:
    np.save(f, predictions)

In [ ]:
!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 1)  # Limit the rating to a maximum of 1 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_all  # Assuming 'bipolar' is the column containing the bipolar disorder labels

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, bipolar_keywords, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_combined, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cross-validation scores:
F1 Score: 0.0348 ± 0.0422
Sensitivity: 0.0198 ± 0.0255
Specificity: 0.9986 ± 0.0028


In [ ]:
with open('bipolar-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

### **ocd**

In [ ]:
ocd_items = [
    "Having to check and double-check what you do.",
    "Having to do things very slowly to insure correctness.",
    "Your mind going blank",
    "Trouble remembering things.",
    "Difficulty making decisions.",
    "Trouble concentrating.",
    "Worried about sloppiness or carelessness.",
    "Feeling blocked in getting things done.",
    "Having to repeat the same actions, i.e., counting, washing."
]

In [ ]:
ocd_items = [
    "check",
    "double-check",
    "slowly",
    "correctness",
    "mind going blank",
    "remembering",
    "trouble remembering",
    "difficulty making decisions",
    "trouble concentrating",
    "worried",
    "sloppiness",
    "carelessness",
    "blocked",
    "getting things done",
    "repeat",
    "same actions",
    "counting",
    "washing"
]

In [ ]:
#test-baseline
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_train = vectorizer.fit_transform(train_data['selected_posts'])
X_test = vectorizer.transform(test_data['selected_posts'])
y_train = y_true_train  # Assuming 'anxiety' is the column containing the anxiety labels in the training data
y_test = y_true_test  # Assuming 'anxiety' is the column containing the anxiety labels in the test data

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and specificity scores
def bootstrap_scores(true_labels, preds):
    tn, fp, fn, tp = confusion_matrix(true_labels, preds).ravel()
    f1 = f1_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    specificity = tn / (tn + fp)
    return f1, recall, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, y_pred) for _ in range(num_bootstraps)]

# Calculate mean and standard deviation of scores
f1_scores, recall_scores, specificity_scores = zip(*bootstrap_scores_list)
mean_f1_score = np.mean(f1_scores)
mean_recall_score = np.mean(recall_scores)
mean_specificity_score = np.mean(specificity_scores)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_specificity_score = np.std(specificity_scores)

# Print results in the desired format
print(f"F1 Score: {mean_f1_score:.4f}±{std_f1_score:.4f}")
print(f"Recall Score: {mean_recall_score:.4f}±{std_recall_score:.4f}")
print(f"Specificity Score: {mean_specificity_score:.4f}±{std_specificity_score:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-29-55aad6a57cac>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))


F1 Score: 0.1957±0.0000
Recall Score: 0.1125±0.0000
Specificity Score: 0.9988±0.0000


In [ ]:
with open('ocd-test-simple.npy', 'wb') as f:
    np.save(f, y_pred)

In [ ]:
#baseline+all
!pip install -U scikit-learn
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords


# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_all


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.1660 ± 0.0370
Sensitivity: 0.1182 ± 0.0520
Specificity: 0.9900 ± 0.0197


In [ ]:
with open('ocd-all-simple.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#test-assessment
#!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 4)  # Limit the rating to a maximum of 4 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, ocd_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the presence of OCD based on the criteria from the checklist
obsessive_threshold = 10
compulsive_threshold = 10
X_ocd = np.where((np.sum(X_ratings[:, :5], axis=1) >= obsessive_threshold) & (np.sum(X_ratings[:, 5:], axis=1) >= compulsive_threshold), 1, 0)

# Combine text features and OCD presence
X_combined = hstack([X_text_vec, X_ocd.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
X_combined = X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

predictions = rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print(f"F1 Score: {np.mean(f1_scores):.4f} ± {std_f1_score:.4f}")
print(f"Sensitivity: {np.mean(recall_scores):.4f} ± {std_recall_score:.4f}")
print(f"Specificity: {np.mean(precision_scores):.4f} ± {std_precision_score:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

F1 Score: 0.0000 ± 0.0000
Sensitivity: 0.0000 ± 0.0000
Specificity: 0.0000 ± 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
with open('ocd-test-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

In [ ]:
#all-assessment
!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 4)  # Limit the rating to a maximum of 4 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train  # Assuming 'ocd' is the column containing the OCD labels

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])
'''
# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, ocd_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the presence of OCD based on the criteria from the checklist
obsessive_threshold = 10
compulsive_threshold = 10
X_ocd = np.where((np.sum(X_ratings[:, :5], axis=1) >= obsessive_threshold) & (np.sum(X_ratings[:, 5:], axis=1) >= compulsive_threshold), 1, 0)

# Combine text features and OCD presence
X_combined = hstack([X_text_vec, X_ocd.reshape(-1, 1)])
'''
# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_text_vec, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 86.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

Cross-validation scores:
F1 Score: 0.0000 ± 0.0000
Sensitivity: 0.0000 ± 0.0000
Specificity: 0.0000 ± 0.0000


In [ ]:
with open('ocd-all-assessment.npy', 'wb') as f:
    np.save(f, cv_scores)

### **Ptsd**

In [ ]:
#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_train = vectorizer.fit_transform(train_data['selected_posts'])
X_test = vectorizer.transform(test_data['selected_posts'])
y_train = y_true_train  # Assuming 'anxiety' is the column containing the anxiety labels in the training data
y_test = y_true_test  # Assuming 'anxiety' is the column containing the anxiety labels in the test data

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and specificity scores
def bootstrap_scores(true_labels, preds):
    tn, fp, fn, tp = confusion_matrix(true_labels, preds).ravel()
    f1 = f1_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    specificity = tn / (tn + fp)
    return f1, recall, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, y_pred) for _ in range(num_bootstraps)]

# Calculate mean and standard deviation of scores
f1_scores, recall_scores, specificity_scores = zip(*bootstrap_scores_list)
mean_f1_score = np.mean(f1_scores)
mean_recall_score = np.mean(recall_scores)
mean_specificity_score = np.mean(specificity_scores)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_specificity_score = np.std(specificity_scores)

# Print results in the desired format
print(f"F1 Score: {mean_f1_score:.4f}±{std_f1_score:.4f}")
print(f"Recall Score: {mean_recall_score:.4f}±{std_recall_score:.4f}")
print(f"Specificity Score: {mean_specificity_score:.4f}±{std_specificity_score:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-108-55aad6a57cac>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


F1 Score: 0.0000±0.0000
Recall Score: 0.0000±0.0000
Specificity Score: 1.0000±0.0000


In [ ]:
with open('ptsd-test-simple.npy', 'wb') as f:
    np.save(f, y_pred)

In [ ]:
#baseline+all

!pip install -U scikit-learn

import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

!python -m nltk.downloader stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_all

# Define the scoring metrics
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [ ]:
# assessment+test
#!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 4)  # Limit the rating to a maximum of 4 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# PTSD assessment items
ptsd_items = [
    'disturbing memories',
    'disturbing dreams',
    'reliving experience',
    'upset by reminders',
    'physical reactions to reminders',
    'avoiding memories',
    'avoiding external reminders',
    'trouble remembering',
    'negative beliefs',
    'blaming',
    'negative feelings',
    'loss of interest',
    'feeling distant',
    'trouble experiencing positive feelings',
    'irritable behavior',
    'taking risks',
    'superalert',
    'easily startled',
    'difficulty concentrating',
    'trouble sleeping'
]

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, ptsd_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the provisional PTSD diagnosis based on the DSM-5 diagnostic rule
X_ptsd = np.zeros(len(X_ratings))
for i, ratings in enumerate(X_ratings):
    b_items = np.sum(ratings[0:5] >= 2) >= 1
    c_items = np.sum(ratings[5:7] >= 2) >= 1
    d_items = np.sum(ratings[7:14] >= 2) >= 2
    e_items = np.sum(ratings[14:20] >= 2) >= 2
    if b_items and c_items and d_items and e_items:
        X_ptsd[i] = 1

# Combine text features and PTSD presence
X_combined = hstack([X_text_vec, X_ptsd.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
X_combined = X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

predictions = rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print(f"F1 Score: {np.mean(f1_scores):.4f} ± {std_f1_score:.4f}")
print(f"Sensitivity: {np.mean(recall_scores):.4f} ± {std_recall_score:.4f}")
print(f"Specificity: {np.mean(precision_scores):.4f} ± {std_precision_score:.4f}")

In [ ]:
# assessment+all
!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(any(keyword.lower() in post.lower() for keyword in item) for post in user_posts)
        rating = min(count, 4)  # Limit the rating to a maximum of 4 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train  # Assuming 'ptsd' is the column containing the PTSD labels

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# PTSD assessment items
ptsd_items = [
    ['intrusive thoughts', 'distressing memories', 'unwanted memories'],
    ['nightmares', 'distressing dreams'],
    ['flashbacks', 'reliving', 'acting as if it is happening again'],
    ['emotional distress', 'upset by reminders', 'psychological distress'],
    ['physical reactions', 'physiological reactions', 'heart pounding', 'sweating', 'shaking'],
    ['avoidance of thoughts', 'avoiding memories', 'avoiding thinking'],
    ['avoidance of reminders', 'avoiding activities', 'avoiding places', 'avoiding people'],
    ['inability to remember', 'difficulty remembering', 'trouble remembering'],
    ['negative beliefs', 'negative thoughts', 'distorted cognitions'],
    ['blaming self', 'blaming others', 'self-blame'],
    ['negative emotions', 'fear', 'horror', 'anger', 'guilt', 'shame'],
    ['diminished interest', 'loss of interest', 'detachment'],
    ['feeling disconnected', 'feeling isolated', 'feeling distant'],
    ['difficulty experiencing positive emotions', 'unable to feel happiness', 'unable to feel love'],
    ['irritability', 'angry outbursts', 'aggression'],
    ['reckless behavior', 'self-destructive behavior', 'taking too many risks'],
    ['hypervigilance', 'being on guard', 'heightened awareness'],
    ['exaggerated startle response', 'easily startled', 'jumpy'],
    ['difficulty concentrating', 'trouble focusing', 'distracted'],
    ['sleep disturbance', 'difficulty falling asleep', 'difficulty staying asleep']
]

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, ptsd_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the provisional PTSD diagnosis based on the DSM-5 diagnostic rule
X_ptsd = np.zeros(len(X_ratings))
for i, ratings in enumerate(X_ratings):
    b_items = np.sum(ratings[0:5] >= 2) >= 1
    c_items = np.sum(ratings[5:7] >= 2) >= 1
    d_items = np.sum(ratings[7:14] >= 2) >= 2
    e_items = np.sum(ratings[14:20] >= 2) >= 2
    if b_items and c_items and d_items and e_items:
        X_ptsd[i] = 1

# Combine text features and PTSD presence
X_combined = hstack([X_text_vec, X_ptsd.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_combined, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

### **Eating**

In [ ]:

#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

!python -m nltk.downloader stopwords
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_train = vectorizer.fit_transform(train_data['selected_posts'])
X_test = vectorizer.transform(test_data['selected_posts'])
y_train = y_true_train  # Assuming 'anxiety' is the column containing the anxiety labels in the training data
y_test = y_true_test  # Assuming 'anxiety' is the column containing the anxiety labels in the test data
# Perform cross-validation
model.fit(X_train, y_train)
predictions=model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<ipython-input-117-8ff7b7adb78b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Standard Deviation of F1 Scores (Bootstrap): 0.0
Standard Deviation of Recall Scores (Bootstrap): 0.0
Standard Deviation of Precision Scores (Bootstrap): 0.0
f1_score 0.0
recall 0.0
precision 1.0
std_f1_score 0.0
std_recall_score 0.0
std_precision_score 0.0


In [ ]:
#baseline+all
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import nltk
nltk.download('stopwords')

# Preprocess the text data
stop_words = set(nltk.corpus.stopwords.words('english'))
# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Split the data into training and testing sets
X = all_data['selected_posts']
y = y_all# Assuming 'eating_disorder' is the column containing the eating disorder labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LinearSVC())
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print(f"F1 Score: {np.mean(f1_scores):.4f} ± {std_f1_score:.4f}")
print(f"Sensitivity: {np.mean(recall_scores):.4f} ± {std_recall_score:.4f}")
print(f"Specificity: {np.mean(precision_scores):.4f} ± {std_precision_score:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


F1 Score: 0.0073 ± 0.0022
Sensitivity: 1.0000 ± 0.0000
Specificity: 0.0037 ± 0.0011


In [ ]:
# Eating disorder assessment items
eating_disorder_items = [
    ['limit food', 'restrict eating', 'influence weight', 'influence shape'],
    ['long periods without eating', 'fasting', 'skipping meals'],
    ['thinking about food', 'preoccupied with food', 'thinking about calories'],
    ['thinking about weight', 'thinking about shape', 'preoccupied with weight', 'preoccupied with shape'],
    ['fear of gaining weight', 'afraid of becoming fat'],
    ['desire to lose weight', 'want to be thinner'],
    ['making yourself sick', 'vomiting', 'taking laxatives'],
    ['excessive exercise', 'driven exercise', 'compulsive exercise'],
    ['loss of control over eating', 'binge eating'],
    ['eating large amounts of food', 'overeating'],
    ['weight influences self-evaluation', 'shape influences self-evaluation']
]

In [ ]:
# assessment-test
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(any(keyword.lower() in post.lower() for keyword in item) for post in user_posts)
        rating = min(count, 4)  # Limit the rating to a maximum of 4 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

def evaluate_rule_based_method(X_test, y_test, assessment_items, stop_words, threshold=14):
    y_pred = []

    for posts in X_test:
        scores = generate_ratings(posts, assessment_items, stop_words)
        total_score = sum(scores)
        predicted_label = 1 if total_score >= threshold else 0
        y_pred.append(predicted_label)

    return y_pred

# Preprocess the text data
stop_words = set(stopwords.words('english'))

X_test=test_data['selected_posts']
X_test = X_test.apply(lambda x: [preprocess_text(post, stop_words) for post in x])
y_test=y_true_test
y_test= pd.factorize(y_test)[0]

# Apply the rule-based method to the test set
y_pred = evaluate_rule_based_method(X_test, y_test, eating_disorder_items, stop_words)

# Number of bootstrap samples
num_bootstraps = 100

# Bootstrap sampling and evaluation
f1_scores = []
precision_scores = []
recall_scores = []

for _ in range(num_bootstraps):
    bootstrap_indices = np.random.choice(len(y_test), size=len(y_test), replace=True)
    y_test_bootstrap = y_test[bootstrap_indices]
    y_pred_bootstrap = [y_pred[i] for i in bootstrap_indices]

    f1 = f1_score(y_test_bootstrap, y_pred_bootstrap)
    precision = precision_score(y_test_bootstrap, y_pred_bootstrap)
    recall = recall_score(y_test_bootstrap, y_pred_bootstrap)

    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)

# Calculate mean and standard deviation of scores
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)
mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

# Print the results
print(f"Mean F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")
print(f"Mean Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f} ± {std_recall:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

Mean F1 Score: 0.0000 ± 0.0000
Mean Precision: 0.0000 ± 0.0000
Mean Recall: 0.0000 ± 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
# assessment-all
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def calculate_eating_disorder_score(user_posts, assessment_items, stop_words):
    scores = []
    for item_list in assessment_items:
        count = sum(any(keyword.lower() in post.lower() for keyword in item_list) for post in user_posts)
        score = min(count, 3)  # Limit the score to a maximum of 3 based on the checklist scale
        scores.append(score)
    return scores

def evaluate_rule_based_method(X, y, assessment_items, stop_words, threshold=14):
    y_pred = []

    for posts in X:
        scores = calculate_eating_disorder_score(posts, assessment_items, stop_words)
        total_score = sum(scores)
        predicted_label = 1 if total_score >= threshold else 0
        y_pred.append(predicted_label)

    return y_pred

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Eating disorder assessment items
eating_disorder_items = [
    ['limit food', 'restrict eating', 'influence weight', 'influence shape'],
    ['long periods without eating', 'fasting', 'skipping meals'],
    ['thinking about food', 'preoccupied with food', 'thinking about calories'],
    ['thinking about weight', 'thinking about shape', 'preoccupied with weight', 'preoccupied with shape'],
    ['fear of gaining weight', 'afraid of becoming fat'],
    ['desire to lose weight', 'want to be thinner'],
    ['making yourself sick', 'vomiting', 'taking laxatives'],
    ['excessive exercise', 'driven exercise', 'compulsive exercise'],
    ['loss of control over eating', 'binge eating'],
    ['eating large amounts of food', 'overeating'],
    ['weight influences self-evaluation', 'shape influences self-evaluation']
]

# Split the data into features (X) and target (y)
X = all_data['selected_posts']
y = y_true_train # Assuming 'eating_disorder' is the column containing the eating disorder labels

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Number of folds for cross-validation
num_folds = 5

# Initialize KFold cross-validator
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Lists to store evaluation scores for each fold
f1_scores = []
precision_scores = []
recall_scores = []

# Perform 5-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Apply the rule-based method to the test fold
    y_pred = evaluate_rule_based_method(X_test, y_test, eating_disorder_items, stop_words)

    # Calculate evaluation scores for the current fold
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)

# Calculate mean and standard deviation of scores across all folds
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)
mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)

# Print the results
print(f"Mean F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")
print(f"Mean Precision: {mean_precision:.4f} ± {std_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f} ± {std_recall:.4f}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

Mean F1 Score: 0.0000 ± 0.0000
Mean Precision: 0.0000 ± 0.0000
Mean Recall: 0.0000 ± 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **schizophrenia**

In [ ]:

#baseline+test
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

!python -m nltk.downloader stopwords
# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Load and preprocess the training data
train_data['selected_posts'] = train_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))
test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X_train = vectorizer.fit_transform(train_data['selected_posts'])
X_test = vectorizer.transform(test_data['selected_posts'])
y_train = y_true_train  # Assuming 'anxiety' is the column containing the anxiety labels in the training data
y_test = y_true_test  # Assuming 'anxiety' is the column containing the anxiety labels in the test data
# Perform cross-validation
model.fit(X_train, y_train)
predictions=model.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<ipython-input-138-8ff7b7adb78b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['selected_posts'] = test_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))


Standard Deviation of F1 Scores (Bootstrap): 0.07430064502958214
Standard Deviation of Recall Scores (Bootstrap): 0.043969943719048575
Standard Deviation of Precision Scores (Bootstrap): 0.0006764067236959868
f1_score 0.13126933859239062
recall 0.0719904587361002
precision 0.9989400921658987
std_f1_score 0.07430064502958214
std_recall_score 0.043969943719048575
std_precision_score 0.0006764067236959868


In [ ]:
#simple+all
import numpy as np
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
!python -m nltk.downloader stopwords

# Load the trained model and vectorizer
model = LogisticRegression(multi_class='multinomial', solver='saga')
vectorizer = TfidfVectorizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to calculate specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    sensitivity = recall_score(bootstrap_true_labels, bootstrap_preds)
    specificity = recall_score(np.logical_not(true_labels) , np.logical_not(bootstrap_preds))
    return f1, sensitivity, specificity


# Load and preprocess the training data
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: ' '.join([word for post in x for word in post.split() if word.lower() not in stop_words]))

# Preprocess the text data
X = vectorizer.fit_transform(all_data['selected_posts'])
y = y_all


# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform cross-validation
cv_scores = cross_validate(model, X, y, cv=5, scoring=scoring, return_estimator=True, return_indices=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])
'''
# Get the predictions for each fold
fold_predictions_cross = []
for estimator, test_index in zip(cv_scores['estimator'], cv_scores['indices']['test']):
    print(test_index)
    X_test_fold = X_combined.tocsr()[test_index]
    y_test_fold = y[test_index]
    y_pred_fold = estimator.predict(X_test_fold)
    fold_predictions_cross.append((y_test_fold, y_pred_fold))
'''
print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cross-validation scores:
F1 Score: 0.1202 ± 0.0527
Sensitivity: 0.0754 ± 0.0360
Specificity: 0.9979 ± 0.0042


In [ ]:
schizophrenia_items = [
    "strange surroundings",
    "unusual sounds",
    "visual disturbances",
    "telepathy",
    "thought control",
    "difficulty communicating",
    "unusually gifted",
    "feeling watched",
    "strange skin sensations",
    "distracted by distant sounds",
    "unseen presence",
    "worry about mental health",
    "feeling non-existent",
    "confusion between real and imaginary",
    "unusual beliefs",
    "changed body parts",
    "loud thoughts",
    "mistrust",
    "unusual visual phenomena",
    "seeing things others can't",
    "difficulty being understood"
]

In [ ]:
# assessment-test
!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

nltk.download('stopwords')

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 1)  # Limit the rating to a maximum of 1 based on the checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train  # Assuming 'schizophrenia' is the column containing the schizophrenia disorder labels

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, schizophrenia_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Combine text features and rating features
X_combined = hstack([X_text_vec, X_ratings])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Define the scoring metrics
scoring = {
    'f1': make_scorer(f1_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(lambda y_true, y_pred: precision_score(y_true, y_pred))
}

# Perform cross-validation
cv_scores = cross_validate(rf_classifier, X_text_vec, y, cv=5, scoring=scoring, return_estimator=True, return_train_score=True)

# Calculate the mean and standard deviation of the scores
f1_mean = np.mean(cv_scores['test_f1'])
f1_std = np.std(cv_scores['test_f1'])
sensitivity_mean = np.mean(cv_scores['test_sensitivity'])
sensitivity_std = np.std(cv_scores['test_sensitivity'])
specificity_mean = np.mean(cv_scores['test_specificity'])
specificity_std = np.std(cv_scores['test_specificity'])

print("Cross-validation scores:")
print(f"F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
print(f"Sensitivity: {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"Specificity: {specificity_mean:.4f} ± {specificity_std:.4f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

Cross-validation scores:
F1 Score: 0.0000 ± 0.0000
Sensitivity: 0.0000 ± 0.0000
Specificity: 0.0000 ± 0.0000


In [ ]:
# assessment-all
#!pip install -U scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

!python -m nltk.downloader stopwords

def preprocess_text(text, stop_words):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def generate_ratings(user_posts, assessment_items, vectorizer, stop_words):
    user_ratings = []
    for item in assessment_items:
        count = sum(item.lower() in post.lower() for post in user_posts)
        rating = min(count, 1)  # Limit the rating to a maximum of 1 based on the schizophrenia checklist scale
        user_ratings.append(rating)
    return user_ratings

# Preprocess the text data
stop_words = set(stopwords.words('english'))
all_data['selected_posts'] = all_data['selected_posts'].apply(lambda x: [preprocess_text(post, stop_words) for post in x])

# Split the data into features (X) and target (y)
X_text = all_data['selected_posts']
y = y_true_train

# Convert the target variable to numerical labels
y = pd.factorize(y)[0]

# Convert text to Bag-of-Words representation
vectorizer = CountVectorizer()
X_text_vec = vectorizer.fit_transform([' '.join(posts) for posts in X_text])

# Generate ratings for each data point
X_ratings = []
for _, row in all_data.iterrows():
    user_posts = row['selected_posts']
    user_ratings = generate_ratings(user_posts, schizophrenia_items, vectorizer, stop_words)
    X_ratings.append(user_ratings)

# Convert ratings to a numpy array
X_ratings = np.array(X_ratings)

# Determine the presence of schizophrenia based on a threshold of 6
schizophrenia_threshold = 6
X_schizophrenia = np.where(np.sum(X_ratings, axis=1) >= schizophrenia_threshold, 1, 0)

# Combine text features and schizophrenia presence
X_combined = hstack([X_text_vec, X_schizophrenia.reshape(-1, 1)])

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
X_combined = X_combined.tocsr()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)
predictions = rf_classifier.predict(X_test)

# Number of bootstrap samples
num_bootstraps = 100

# Function to calculate F1, recall, and precision scores
def bootstrap_scores(true_labels, preds):
    indices = np.random.choice(len(true_labels), len(true_labels), replace=True)
    bootstrap_true_labels = np.take(true_labels, indices)
    bootstrap_preds = np.take(preds, indices)
    f1 = f1_score(bootstrap_true_labels, bootstrap_preds)
    recall = recall_score(bootstrap_true_labels, bootstrap_preds)
    precision = precision_score(bootstrap_true_labels, bootstrap_preds)
    return f1, recall, precision

# Generate bootstrap samples and calculate scores
bootstrap_scores_list = [bootstrap_scores(y_test, predictions) for _ in range(num_bootstraps)]

# Calculate standard deviation of scores
f1_scores, recall_scores, precision_scores = zip(*bootstrap_scores_list)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

# Print results
print("Standard Deviation of F1 Scores (Bootstrap):", std_f1_score)
print("Standard Deviation of Recall Scores (Bootstrap):", std_recall_score)
print("Standard Deviation of Precision Scores (Bootstrap):", std_precision_score)

# Print results
print("f1_score", np.mean(f1_scores))
print("recall", np.mean(recall_scores))
print("precision", np.mean(precision_scores))
print("std_f1_score", std_f1_score)
print("std_recall_score", std_recall_score)
print("std_precision_score", std_precision_score)

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

Standard Deviation of F1 Scores (Bootstrap): 0.0
Standard Deviation of Recall Scores (Bootstrap): 0.0
Standard Deviation of Precision Scores (Bootstrap): 0.0
f1_score 0.0
recall 0.0
precision 0.0
std_f1_score 0.0
std_recall_score 0.0
std_precision_score 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p